<a href="https://colab.research.google.com/github/korayaslaan/artificial_intelligence/blob/main/yemeksepetiyorumlar_ipynd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [168]:
import os
import bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import time
import urllib.request
import re
import urllib3
from pandas import DataFrame
import csv
import datetime
from datetime import datetime, timedelta
import os
import csv
import concurrent
import multiprocessing
from multiprocessing import pool
import io
from pprint import pprint

In [170]:
from bs4.element import Comment
def creator(url):
    headers = requests.utils.default_headers()
    headers.update({
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
    })
    r = requests.get(url, headers=headers)   
    soup = BeautifulSoup(r.content, 'html5lib')
    datas = soup.getText() #gelen verideki html taglarını kaldırıp sadece text kısmını aldım

    temp = datas
    tempData = temp.split("pageKey") #bir sonraki sayfaya erişim işin her request sonunda bi key dönüyo onu aldum 
    pageKey = tempData[1];

    unlike = '":,}'
    for char in unlike:
      pageKey = pageKey.replace(char,"") #buralarda istenmeyen karakterleri silerek daha temiz bir  yorum çektim

  
    print("***************************************************************************************************************************************************************************")
    print("NextPageKey=  " + pageKey)
    nextUrl = "https://reviews-api-tr.fd-api.com/reviews/vendor/ziwm?global_entity_id=YS_TR&limit=30&nextPageKey=" + pageKey
    print("NextPageUrl=  "  + nextUrl)
    print("***************************************************************************************************************************************************************************")
    #kullanıcıyı bilgilendirmek için burada bir sonraki sayfanın linki oluşturuluyor ve key ekrana basılıyor

    #kazınacak mağaza kodu !!!!https://reviews-api-tr.fd-api.com/reviews/vendor/ziwm?global_entity_id=YS_TR&limit=30&!!!! url içindeki 
    #https://reviews-api-tr.fd-api.com/reviews/vendor/ dan sonra yazan 4 haneli kod bu kod mağazaya tıklandığında url den görünüyor zaten 
    #hem burada değişilecek hemde aşağıdaki creator çağrı fonksiyonunda değişcek

    datas = datas.split("text") #yorumların değişken adı text ile başlıyor tüm textleri alırsam tüm yorumları almış olurum
    lenDatas = len(datas)
    for i in range(1,lenDatas):#kaç tane yorum var ise o kadar dönüyor ama buna mağaza yorumlarıda dahil
      comment = datas[i].split("isAnonymous");
      userComment = comment[0]
      unlikeTwo = '":,}' #istemediğimiz işaretleri temizliyoruz 
      for char in unlikeTwo:
        userComment = userComment.replace(char,"")
      if(len(userComment)<2): #otomarik iptallerde metin verilmediği için frontend tarafında görünen yazı  aşağıda değişkene atılıyor
        userComment = "Sipariş ortalama teslimat süresini aştığı için kullanıcının talebiyle iptal edilmiştir. (Bu Yemeksepeti tarafından yapılan otomatik bir değerlendirmedir)" 
      control = userComment.find("createdAt") #mağaza eğer cevap verdiyse yoruma bunun değeri 1 den büyük oluyor ve aşağıdaki if e girip kendi döngüsünü kırıyor
      if(control>1):
        continue #break kullanırsam fonksiyon durur continue sadece kendi döngüsünü kırar sonraki değerden devam eder
      with open("Comments.txt", 'a') as file: #dosyaya yorumları yazıyor
       file.write(userComment+'\n')
       file.close()
      print("CommentUser " + " = " + userComment) #consola yazıyor
    if(len(pageKey)>10): #eğer pagekey oluşursa requestte demekki sonraki sayfa var. Sonraki sayfa varsa metot kendini tekrar çağırır ve tekrara düşer(recursive)
      creator(nextUrl)
    else: #eğer pagekey yok ise son requesti almışızdır ve fonksiyonumuz bitebilir
      print("************************************************")


In [171]:
creator("https://reviews-api-tr.fd-api.com/reviews/vendor/ziwm?global_entity_id=YS_TR&limit=30")
#linkte sadece https://reviews-api-tr.fd-api.com/reviews/vendor/     dan sonraki 4 hane değişcek soru işaretine kadar yani o mağaza id si diyebiliriz
#hangi mağazanın yorumlarını çekeceksen ordaki 4 haneyi burada ve yukarda ki url yi değiştirip çalıştıracaksın

***************************************************************************************************************************************************************************
NextPageKey=  1648165705$$557610FC-D590-455E-9EF1-A8178316BD7F

NextPageUrl=  https://reviews-api-tr.fd-api.com/reviews/vendor/ziwm?global_entity_id=YS_TR&limit=30&nextPageKey=1648165705$$557610FC-D590-455E-9EF1-A8178316BD7F

***************************************************************************************************************************************************************************
CommentUser  = cokk kokuyordu yesilligi 10 gunluktu artik sararip kurumustu yesillik   
CommentUser  = Eh işte. 
CommentUser  = Alışkın olduğum tarzda bir çiğ köfte değildi. Biraz daha tane tane değilde bu tarzı seviyorsanız yenebilir. ekmekleri gerçekten ayrıca lezzetliydi. Ellerinize sağlık  
CommentUser  = başarılı elinize emeğinize sağlık  
CommentUser  = Hayatimda yediğim en kötü çiğköfte 
CommentUser  = kötü  
CommentUse

In [ ]:
# https://reviews-api-tr.fd-api.com/reviews/vendor/wrxv?global_entity_id=YS_TR&limit=30&nextPageKey=1652900124$$c27aa881-6703-4bae-b2ef-30028fd5f45a

In [ ]:
LinksNews=[]
with open("Nytimes.txt",'r',newline='') as f:
  for i in f.readlines():    
    LinksNews.append(i)

t1=time.time()
with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(creator,i.strip()) for i in LinksNews]

print(time.time()-t1)
